In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
NSE Option Chain Sentiment (with yfinance price-based sentiment)
----------------------------------------------------------------
• Reads tickers from symbols.txt
• Fetches option chain from NSE
• Fetches previous close from Yahoo Finance
• Uses Price + OI quadrant logic:
    - Long Build-up
    - Short Build-up
    - Short Covering
    - Long Unwinding
• Saves one fresh summary.csv each run
"""

from __future__ import annotations
import os, time, math, re, datetime as dt
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import pandas as pd, requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import yfinance as yf

# ---------------- CONFIG ----------------
class CONFIG:
    SYMBOLS_FILE = "symbols.txt"
    OUTPUT_DIR = "option_chain_outputs"
    EXPIRY_PREF = "auto"       # "auto" / "weekly" / "monthly" / "YYYY-MM-DD"
    TOTAL_RETRIES = 3
    BACKOFF_FACTOR = 0.35
    REQUEST_SLEEP_SEC = 1.0

NSE_HOST = "https://www.nseindia.com"
OC_ENDPOINTS = {
    "indices": NSE_HOST + "/api/option-chain-indices?symbol={symbol}",
    "equities": NSE_HOST + "/api/option-chain-equities?symbol={symbol}",
}
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) Chrome/120 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": NSE_HOST + "/option-chain",
}
MONTH_MAP = dict(JAN=1,FEB=2,MAR=3,APR=4,MAY=5,JUN=6,JUL=7,AUG=8,SEP=9,OCT=10,NOV=11,DEC=12)
DATE_PAT = re.compile(r"(\d{1,2})-([A-Za-z]{3})-(\d{4})")

# ---------- HTTP client ----------
def build_session(retries, backoff):
    s = requests.Session()
    s.headers.update(HEADERS)
    retry = Retry(
        total=retries,
        backoff_factor=backoff,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=("GET", "HEAD"),
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

class NSEClient:
    def __init__(self):
        self.s = build_session(CONFIG.TOTAL_RETRIES, CONFIG.BACKOFF_FACTOR)

    def boot(self):
        self.s.get(NSE_HOST, timeout=10)

    def fetch(self, sym, seg):
        self.boot()
        url = OC_ENDPOINTS[seg].format(symbol=sym)
        r = self.s.get(url, timeout=15)
        if r.status_code != 200:
            raise RuntimeError(f"{sym}: HTTP {r.status_code}")
        return r.json()

# ---------- utilities ----------
def parse_exp(s: str) -> dt.date:
    m = DATE_PAT.fullmatch(s.strip())
    if m:
        return dt.date(
            int(m.group(3)),
            MONTH_MAP[m.group(2).upper()[:3]],
            int(m.group(1)),
        )
    # assume ISO format
    return dt.date.fromisoformat(s)

def classify_exp(d: dt.date) -> str:
    first_next = (d.replace(day=1) + dt.timedelta(days=32)).replace(day=1)
    last = first_next - dt.timedelta(days=1)
    t = last
    while t.weekday() != 3:  # Thursday
        t -= dt.timedelta(days=1)
    return "monthly" if d == t else "weekly"

def choose_exp(exp: List[str], pref: str) -> str:
    if not exp:
        raise ValueError("No expiries available from NSE payload")

    if pref not in {"auto", "weekly", "monthly"}:
        # pref is specific date
        wanted = parse_exp(pref)
        for e in exp:
            if parse_exp(e) == wanted:
                return e
        return exp[0]

    dated = sorted((parse_exp(e), e) for e in exp)
    today = dt.date.today()
    for d, e in dated:
        if d >= today:
            if pref == "auto" or classify_exp(d) == pref:
                return e
    return dated[0][1]

def infer_segment(sym: str) -> str:
    return "indices" if sym.upper() in {"NIFTY", "BANKNIFTY", "FINNIFTY"} else "equities"

# yfinance mapping for indices
YF_INDEX_MAP = {
    "NIFTY": "^NSEI",
    "BANKNIFTY": "^NSEBANK",
    # FINNIFTY mapping can be adjusted if you know the exact Yahoo symbol
    "FINNIFTY": "^NSEFIN",
}

def get_prev_close(symbol: str) -> Optional[float]:
    sym_u = symbol.upper()
    if sym_u in YF_INDEX_MAP:
        yf_symbol = YF_INDEX_MAP[sym_u]
    else:
        yf_symbol = sym_u + ".NS"

    try:
        data = yf.Ticker(yf_symbol).history(period="3d")
        if len(data) >= 2:
            # second last close = previous close
            return float(data["Close"].iloc[-2])
        elif len(data) == 1:
            return float(data["Close"].iloc[0])
    except Exception:
        pass
    return None

# ---------- core transforms ----------
def to_df(raw) -> pd.DataFrame:
    recs = raw.get("records", {}).get("data", [])
    rows = []
    for n in recs:
        s = n.get("strikePrice")
        exp = n.get("expiryDate")
        ce = n.get("CE", {}) or {}
        pe = n.get("PE", {}) or {}
        rows.append(dict(
            strike=s,
            expiry=exp,
            ce_oi=ce.get("openInterest", 0),
            ce_coi=ce.get("changeinOpenInterest", 0),
            pe_oi=pe.get("openInterest", 0),
            pe_coi=pe.get("changeinOpenInterest", 0),
        ))
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    for c in ["strike", "ce_oi", "ce_coi", "pe_oi", "pe_coi"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    return df.dropna(subset=["expiry"])

@dataclass
class Report:
    symbol: str
    segment: str
    expiry: str
    asof: str

    underlying: Optional[float]
    prev_close: Optional[float]
    price_change_pct: Optional[float]
    price_dir: str  # "Up", "Down", "Flat"

    pcr: float
    total_call_oi: int
    total_put_oi: int
    delta_call_oi: int
    delta_put_oi: int
    total_delta_oi: int
    quadrant: str  # Long Build-up / Short Build-up / Short Covering / Long Unwinding / Indeterminate

    call_top5: List[Tuple[int, int]]
    put_top5: List[Tuple[int, int]]
    ce_hot: Optional[Tuple[int, int]]
    pe_hot: Optional[Tuple[int, int]]
    res_gaps: List[int]
    sup_gaps: List[int]
    equilibrium: bool
    eq_strike: Optional[int]
    sentiment: str
    rationale: str

def _topn(s: pd.Series, n: int):
    s = s[s > 0].sort_values(ascending=False)
    return [(int(k), int(v)) for k, v in s.head(n).items()]

def _gaps(vals: List[int], k=3):
    if len(vals) < 2:
        return []
    u = sorted(set(vals))
    gaps = [u[i + 1] - u[i] for i in range(len(u) - 1) if u[i + 1] > u[i]]
    return sorted(gaps)[:k]

# ---------- quadrant classification ----------
def classify_quadrant(price_dir: str, d_call: int, d_put: int) -> Tuple[str, int]:
    """
    Uses total ΔOI (d_call + d_put) vs price direction.

    Rules:
        Price ↑ & ΔOI > 0 → Long Build-up
        Price ↓ & ΔOI > 0 → Short Build-up
        Price ↑ & ΔOI < 0 → Short Covering
        Price ↓ & ΔOI < 0 → Long Unwinding
        Else              → Indeterminate
    """
    total = d_call + d_put
    if price_dir == "Flat" or total == 0:
        return "Indeterminate", total

    if price_dir == "Up":
        if total > 0:
            return "Long Build-up", total
        elif total < 0:
            return "Short Covering", total
    elif price_dir == "Down":
        if total > 0:
            return "Short Build-up", total
        elif total < 0:
            return "Long Unwinding", total

    return "Indeterminate", total

# ---------- sentiment engine ----------
def derive_sentiment(
    pcr: float,
    d_put: int,
    d_call: int,
    price_dir: str,
    quadrant: str,
    total_delta_oi: int,
):
    score = 0
    reason: List[str] = []

    # PCR logic
    if pcr < 0.8:
        score -= 1
        reason.append(f"PCR {pcr:.2f} (<0.80) → bearish tilt")
    elif pcr > 1.2:
        score += 1
        reason.append(f"PCR {pcr:.2f} (>1.20) → bullish tilt")
    else:
        reason.append(f"PCR {pcr:.2f} ~neutral")

    # Net call vs put flow
    if d_put > d_call:
        score += 1
        reason.append(
            f"ΔPutOI {d_put:,} > ΔCallOI {d_call:,} → put-side strength (bullish tilt)"
        )
    elif d_call > d_put:
        score -= 1
        reason.append(
            f"ΔCallOI {d_call:,} > ΔPutOI {d_put:,} → call-side pressure (bearish tilt)"
        )
    else:
        reason.append("ΔOI balanced between calls and puts")

    # Price + OI logic (qualitative)
    if price_dir == "Up":
        reason.append("Price is UP → underlying bullish bias")
    elif price_dir == "Down":
        reason.append("Price is DOWN → underlying bearish bias")
    else:
        reason.append("Price is FLAT → sentiment driven by OI/PCR")

    # Quadrant logic (explicit label + score tweak)
    if quadrant == "Long Build-up":
        score += 1
        reason.append(
            f"Quadrant: Long Build-up (Price ↑, ΔTotal OI {total_delta_oi:+,}) → strong bullish continuation"
        )
    elif quadrant == "Short Build-up":
        score -= 1
        reason.append(
            f"Quadrant: Short Build-up (Price ↓, ΔTotal OI {total_delta_oi:+,}) → strong bearish continuation"
        )
    elif quadrant == "Short Covering":
        score += 1
        reason.append(
            f"Quadrant: Short Covering (Price ↑, ΔTotal OI {total_delta_oi:+,}) → bullish reversal/relief rally"
        )
    elif quadrant == "Long Unwinding":
        score -= 1
        reason.append(
            f"Quadrant: Long Unwinding (Price ↓, ΔTotal OI {total_delta_oi:+,}) → bearish reversal/profit booking"
        )
    else:
        reason.append(
            f"Quadrant: Indeterminate (Price {price_dir}, ΔTotal OI {total_delta_oi:+,})"
        )

    sentiment = "Bullish" if score > 0 else "Bearish" if score < 0 else "Neutral"
    return sentiment, "; ".join(reason)

# ---------- analysis ----------
def analyze(raw, sym: str, seg: str, pref: str) -> Report:
    recs = raw.get("records", {})

    underlying = recs.get("underlyingValue")
    prev_close = get_prev_close(sym)

    price_change_pct: Optional[float] = None
    price_dir = "Flat"

    if isinstance(underlying, (int, float)) and isinstance(prev_close, (int, float)) and prev_close != 0:
        price_change_pct = (underlying - prev_close) / prev_close * 100.0
        if price_change_pct > 0.10:
            price_dir = "Up"
        elif price_change_pct < -0.10:
            price_dir = "Down"
        else:
            price_dir = "Flat"

    exp = choose_exp(recs.get("expiryDates", []), pref)
    df = to_df(raw)
    if df.empty:
        raise ValueError(f"{sym}: Empty option-chain dataframe after transform")

    dfe = df[df["expiry"] == exp]

    tce = int(dfe["ce_oi"].sum())
    tpe = int(dfe["pe_oi"].sum())
    pcr = (tpe / tce) if tce else math.inf

    dce = int(dfe["ce_coi"].sum())
    dpe = int(dfe["pe_coi"].sum())
    total_delta_oi = dce + dpe

    ce_agg = _topn(dfe.groupby("strike")["ce_oi"].sum(), 5)
    pe_agg = _topn(dfe.groupby("strike")["pe_oi"].sum(), 5)

    ce_flow = dfe.groupby("strike")["ce_coi"].sum().sort_values(ascending=False)
    pe_flow = dfe.groupby("strike")["pe_coi"].sum().sort_values(ascending=False)
    ce_hot = (int(ce_flow.index[0]), int(ce_flow.iloc[0])) if not ce_flow.empty else None
    pe_hot = (int(pe_flow.index[0]), int(pe_flow.iloc[0])) if not pe_flow.empty else None

    quadrant, total_delta_oi = classify_quadrant(price_dir, dce, dpe)
    sentiment, rationale = derive_sentiment(
        pcr, dpe, dce, price_dir, quadrant, total_delta_oi
    )

    eq = False
    eqs = None
    if ce_agg and pe_agg and ce_agg[0][0] == pe_agg[0][0]:
        eq = True
        eqs = ce_agg[0][0]

    return Report(
        symbol=sym,
        segment=seg,
        expiry=exp,
        asof=recs.get("timestamp", ""),
        underlying=underlying,
        prev_close=prev_close,
        price_change_pct=price_change_pct,
        price_dir=price_dir,
        pcr=pcr,
        total_call_oi=tce,
        total_put_oi=tpe,
        delta_call_oi=dce,
        delta_put_oi=dpe,
        total_delta_oi=total_delta_oi,
        quadrant=quadrant,
        call_top5=ce_agg,
        put_top5=pe_agg,
        ce_hot=ce_hot,
        pe_hot=pe_hot,
        res_gaps=_gaps([s for s, _ in ce_agg]),
        sup_gaps=_gaps([s for s, _ in pe_agg]),
        equilibrium=eq,
        eq_strike=eqs,
        sentiment=sentiment,
        rationale=rationale,
    )

# ---------- formatting ----------
def fmt_pairs(p): return "; ".join(f"{s}:{oi}" for s, oi in p)
def fmt_list(l): return "; ".join(map(str, l))

def build_report(r: Report) -> str:
    L: List[str] = []
    L.append("=" * 72)
    L.append(f"Symbol: {r.symbol} | Segment: {r.segment} | Expiry: {r.expiry}")
    L.append(f"As Of : {r.asof}")
    if r.underlying is not None:
        line = f"Underlying: {r.underlying:.2f}"
        if r.prev_close is not None:
            line += f" | Prev Close: {r.prev_close:.2f}"
        if r.price_change_pct is not None:
            line += f" | Move: {r.price_dir} ({r.price_change_pct:+.2f}%)"
        else:
            line += f" | Move: {r.price_dir}"
        L.append(line)
    L.append("-" * 72)
    L.append(f"Total Call OI : {r.total_call_oi:,}")
    L.append(f"Total Put  OI : {r.total_put_oi:,}")
    L.append(f"PCR           : {r.pcr:.2f}" if math.isfinite(r.pcr) else f"PCR           : inf")
    L.append(f"ΔCall OI      : {r.delta_call_oi:+,}")
    L.append(f"ΔPut  OI      : {r.delta_put_oi:+,}")
    L.append(f"ΔTotal OI     : {r.total_delta_oi:+,} | Quadrant: {r.quadrant}")
    L.append("-" * 72)
    L.append("Top-5 Resistances (CE OI): " + fmt_pairs(r.call_top5))
    L.append("Top-5 Supports    (PE OI): " + fmt_pairs(r.put_top5))
    if r.ce_hot:
        L.append(f"Flow (ΔOI) CE hotspot: {r.ce_hot[0]} | ΔOI: {r.ce_hot[1]:+,}")
    if r.pe_hot:
        L.append(f"Flow (ΔOI) PE hotspot: {r.pe_hot[0]} | ΔOI: {r.pe_hot[1]:+,}")
    if r.res_gaps:
        L.append(f"Closest resistance gaps: {fmt_list(r.res_gaps)}")
    if r.sup_gaps:
        L.append(f"Closest support gaps   : {fmt_list(r.sup_gaps)}")
    if r.equilibrium:
        L.append("-" * 72)
        L.append(f"⚪ Equilibrium Zone: {r.eq_strike}")
    L.append("-" * 72)
    L.append(f"Sentiment: {r.sentiment}")
    L.append(f"Why      : {r.rationale}")
    L.append("=" * 72)
    return "\n".join(L)

# ---------- main ----------
if __name__ == "__main__":
    os.makedirs(CONFIG.OUTPUT_DIR, exist_ok=True)
    client = NSEClient()

    with open(CONFIG.SYMBOLS_FILE) as f:
        syms = [s.strip().upper() for s in f if s.strip()]

    rows = []
    for sym in syms:
        seg = infer_segment(sym)
        try:
            raw = client.fetch(sym, seg)
            rep = analyze(raw, sym, seg, CONFIG.EXPIRY_PREF)
            text = build_report(rep)
            print(text)
            rows.append(dict(
                symbol=rep.symbol,
                segment=rep.segment,
                expiry=rep.expiry,
                as_of=rep.asof,
                underlying=rep.underlying,
                prev_close=rep.prev_close,
                price_change_pct=rep.price_change_pct,
                price_dir=rep.price_dir,
                pcr=rep.pcr,
                total_call_oi=rep.total_call_oi,
                total_put_oi=rep.total_put_oi,
                delta_call_oi=rep.delta_call_oi,
                delta_put_oi=rep.delta_put_oi,
                total_delta_oi=rep.total_delta_oi,
                quadrant=rep.quadrant,
                top5_resistances=fmt_pairs(rep.call_top5),
                top5_supports=fmt_pairs(rep.put_top5),
                res_gaps=fmt_list(rep.res_gaps),
                sup_gaps=fmt_list(rep.sup_gaps),
                ce_hotspot=rep.ce_hot,
                pe_hotspot=rep.pe_hot,
                equilibrium=rep.equilibrium,
                equilibrium_strike=rep.eq_strike,
                sentiment=rep.sentiment,
                rationale=rep.rationale,
                report_text=text,
                status="ok",
            ))
        except Exception as e:
            print(f"[WARN] {sym}: {e}")
            rows.append(dict(symbol=sym, segment=seg, status=f"error: {e}"))
        time.sleep(CONFIG.REQUEST_SLEEP_SEC)

    summary_path = os.path.join(CONFIG.OUTPUT_DIR, "summary.csv")
    pd.DataFrame(rows).to_csv(summary_path, index=False)
    print(f"\n✅ Summary saved to {summary_path} (fresh each run)")


Symbol: HINDUNILVR | Segment: equities | Expiry: 25-Nov-2025
As Of : 18-Nov-2025 11:03:48
Underlying: 2411.70 | Prev Close: 2425.00 | Move: Down (-0.55%)
------------------------------------------------------------------------
Total Call OI : 36,925
Total Put  OI : 15,447
PCR           : 0.42
ΔCall OI      : -239
ΔPut  OI      : -142
ΔTotal OI     : -381 | Quadrant: Long Unwinding
------------------------------------------------------------------------
Top-5 Resistances (CE OI): 2500:7065; 2600:5957; 2700:2640; 2520:2388; 2460:2163
Top-5 Supports    (PE OI): 2500:2353; 2400:1704; 2300:1362; 2440:1309; 2600:1284
Flow (ΔOI) CE hotspot: 2400 | ΔOI: +426
Flow (ΔOI) PE hotspot: 2400 | ΔOI: +55
Closest resistance gaps: 20; 40; 80
Closest support gaps   : 40; 60; 100
------------------------------------------------------------------------
⚪ Equilibrium Zone: 2500
------------------------------------------------------------------------
Sentiment: Bearish
Why      : PCR 0.42 (<0.80) → bearish t